In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
(X_train,y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
classes = ['airplane','automobile','bird','cat','deer','dog','frog', 'horse','ship','truck']

In [6]:
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [7]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [8]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [9]:
y_train[0:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [10]:
y_train_categorical[0:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [11]:
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [12]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [13]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [14]:
strategy.num_replicas_in_sync

1

In [15]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [16]:
%%timeit -n1 -r1 
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=5)

Epoch 1/5
200/200 [==============================] - 105s 489ms/step - loss: 1.9768 - accuracy: 0.2962
Epoch 2/5
200/200 [==============================] - 79s 394ms/step - loss: 1.8151 - accuracy: 0.3635
Epoch 3/5
200/200 [==============================] - 79s 396ms/step - loss: 1.7534 - accuracy: 0.3878
Epoch 4/5
200/200 [==============================] - 101s 503ms/step - loss: 1.7104 - accuracy: 0.4051
Epoch 5/5
200/200 [==============================] - 79s 395ms/step - loss: 1.6761 - accuracy: 0.4190
7min 41s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [17]:
%%timeit -n1 -r1 
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=5)

Epoch 1/5
200/200 [==============================] - 90s 449ms/step - loss: 1.9896 - accuracy: 0.2958
Epoch 2/5
200/200 [==============================] - 82s 410ms/step - loss: 1.8233 - accuracy: 0.3628
Epoch 3/5
200/200 [==============================] - 83s 413ms/step - loss: 1.7609 - accuracy: 0.3868
Epoch 4/5
200/200 [==============================] - 78s 392ms/step - loss: 1.7173 - accuracy: 0.4021
Epoch 5/5
200/200 [==============================] - 83s 416ms/step - loss: 1.6826 - accuracy: 0.4150
7min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
